In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
from tqdm import tqdm
import re
from abnumber import Chain
import json
from urllib.parse import urlencode
import concurrent.futures

import seaborn as sns
import matplotlib.pyplot as plt

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

#%matplotlib inline


In [12]:
# Define Sample Path, Define Humanized Path
sample_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/sample_humanization_result.csv'
humanized_fpath = '/data/home/waitma/antibody_proj/antidiff/data/lab_data/humanization_pair_data_filter.csv'

In [13]:
# Get DataFrame from the fpath.
sample_df = pd.read_csv(sample_fpath)
humanized_df = pd.read_csv(humanized_fpath)

In [5]:
# Define deal out-of-order table.
def regular_order_table(out_of_order_table):
    all_table_data = []
    for table in out_of_order_table:
        table_data = []
        for row in table.find_all('tr'):
            row_data = []
            for cell in row.find_all(['th', 'td']):
                row_data.append(cell.text)
            table_data.append(row_data)
        all_table_data.append(table_data)
    return all_table_data[:2]  # only the first two will be used, all is three.

# Define extract data. Only want to know wther the sequence can be viewed as human.
def extract_human_data(regular_table):
    extracted_data = []
    for table_data in regular_table:
        table_header = table_data[0]
        human_row = [None, None, None, None]
        for row in table_data:
            if row[-1] == 'HUMAN':
                human_row = row
        extracted_data.extend(human_row)
    return extracted_data

# Define request process.
def get_predict_result(job_name, h_seq, l_seq):
    # Url path
    humab_url = 'https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab'

    data = {
        'h_sequence_score': h_seq,
        'l_sequence_score': l_seq,
        'jobname_score': job_name
    }
    reponse = requests.post(humab_url, data=data)
    result_url = reponse.url
    print(result_url)

    # Need to wait a moment.
    time.sleep(15)

    # Get the result page.
    result_response = requests.get(result_url)

    if result_response.status_code == 200:
        soup = BeautifulSoup(result_response.text, 'html.parser')
        tables = soup.find_all('table', {'class': 'table table-results'})
        # print(tables)
        
        predict_table = regular_order_table(tables)
        print(predict_table)
        extract_data = extract_human_data(predict_table)
        print(extract_data)
    else:
        print('May be url has problem or need larger sleep time.')

    sequence_list = [h_seq, l_seq]
    return extract_data + sequence_list
    

In [6]:
# Get the sample sequence from data.
# Correct dataframe name
new_columns_name = {
    'Specific': 'Specific',
    'name': 'name',
    'HSEQ': 'hseq',
    'LSEQ': 'lseq'
}

sample_df = sample_df.rename(columns=new_columns_name)
sample_human_df = sample_df[sample_df['Specific'] == 'humanization'].reset_index(drop=True)
sample_human_df.head()

,Specific,name,hseq,lseq,Unnamed: 4
0,humanization,3A3human_0,EVQLVESGGGLVQPGGSLRLSCVASGFTFSNFWMNWVRQAPGKGLE...,DIQMTQSPSPLSASVGDRANITCRASQDVSTDLGWYQQKPGKAPKV...,NaN
1,humanization,MAK195human_0,QVQLVESGGGVVQPGGSLRLSCAASGFSLTDYGVAWVRQPPGKGLE...,DIQMAQSPSSLSASVGDRVTITCRASQAVSSALTWYQQIPGKSPKL...,NaN
2,humanization,31#human_0,EVQLVQSGAEVKKPGESLKISCKASGYTFTDYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATISCKSSQSLLNTNSQKNYIAWYQQKP...,NaN
3,humanization,56#human_0,QVQLQESGPRLVEPSETLSLTCTVSGYSITSDYDWGWIRQPPGRGL...,DTLLTQSPDHLSVTLGERATISCRASQNIGTSLNWYQQKPGQSPKR...,NaN
4,humanization,74#human_0,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...,NaN


In [7]:
filter_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/filter_humanization_result.csv'
filter_human_df = pd.read_csv(filter_fpath)
filter_human_df

,Specific,name,hseq,lseq,Unnamed: 4
0,humanization,3A3human_0,EVQLVESGGGLVQPGGSLRLSCVASGFTFSNFWMNWVRQAPGKGLE...,DIQMTQSPSPLSASVGDRANITCRASQDVSTDLGWYQQKPGKAPKV...,NaN
1,humanization,31#human_0,EVQLVQSGAEVKKPGESLKISCKASGYTFTDYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATISCKSSQSLLNTNSQKNYIAWYQQKP...,NaN
2,humanization,74#human_0,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...,NaN
3,humanization,SC73.38human_0,QVHLVQSGAEVKKPGASVKVSCKASGYSFTGYTIHWVRQAPGQGLE...,EIVMTQSPATLSVSPGETATLSCRASQSISNNLVWYQQKPDQAPRL...,NaN
4,humanization,SC73.39human_0,QVQLVQSGAEVKKPGASVKVSCRTSGYSFTNYNFAWVRQAPRQGLE...,DIVMTQSPDSLAVSLGERATIKCKSSQSLLNSSNQKNYLAWYQQKP...,NaN
...,...,...,...,...,...
111,humanization,RK22human_0,EVQLVQSGVEVKKPGESLKISCKGSGYTFTSYWIGWVRQMPGKGLE...,DIVVTQSPDSLAVSLGERATINCKSSQSLLNSANQKNYLAWYQQKP...,NaN
112,humanization,49G8human_0,QVQLVQSGSEVKKPGASVKVSCQASGYTFADYEIHWMRQAPGQGPE...,DVVVTQSPLSLPVTLGQPASISCRSSQSIVHSNGNTYLDWFQQRPG...,NaN
113,humanization,mu515H7human_0,EVQLVESGGGLVQPGGSLRLSCATSGFTFTDNYMDWVRQAPGKGLE...,IVLTQSPDSLAVSLGERATINCKSSQSLFNSRTRKNYLTWYQQKPG...,NaN
114,humanization,mu16B5human_0,QVRLVQSGAEVKKPGASVGVSCKASGFNIKDIYIHWVRQVPGQGPE...,DIVMTQFPNSLAVSLGERATINCKSSQSLLNSRTRKNYLSWYQKKA...,NaN


In [18]:
T20_REGEX = re.compile('<td>T20 Score:</td><td>([0-9.]+)</td>')
def get_t20_online(seq):
    chain = Chain(seq, scheme='imgt')
    chain_type = 'vh' if chain.chain_type == 'H' else ('vl' if chain.chain_type == 'L' else 'vk')
    html = None
    for retry in range(5):
        url = f'https://sam.curiaglobal.com/t20/cgi-bin/blast.py?chain={chain_type}&region=1&output=3&seqs={seq}'
        try:
            request = requests.get(url)
            if request.ok:
                html = request.text
                break
        except Exception as e:
            print(e)
        time.sleep(0.5 + retry * 5)
        print('Retry', retry+1)
    if not html:
        sys.exit(1)
    # print(html)
    matches = T20_REGEX.findall(html)
    time.sleep(1)
    if not matches:
        print(html)
        # raise ValueError(f'Error calling url {url}')
        return None, None
    return float(matches[0]), chain_type

def get_pair_data_t20(h_seq, l_seq):
    h_score, h_type = get_t20_online(h_seq)
    l_score, l_type = get_t20_online(l_seq)
    print(h_score, l_score)
    return [h_score, h_type, l_score, l_type, h_seq, l_seq]

def process_line(line):
    h_seq = line[1]['h_seq']
    l_seq = line[1]['l_seq']
    name = [line[1]['name']]
    for retry in range(10):
        try:
            data = get_pair_data_t20(h_seq, l_seq)
            if len(data) > 2:
                break
        except:
            time.sleep(5)
            continue
    if len(data) != 2:
        new_data = name + data
        new_line_df = pd.DataFrame([new_data], columns=t20_filter_df.columns)
        return new_line_df
    else:
        return None

In [9]:
# Define a new dataframe for saving results.
# t20_df = pd.DataFrame(columns=['Raw_name', 'H_Score', 'H_gene', 'L_score', 'L-gene', 'H_seq', 'L_seq'])

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     results = list(tqdm(executor.map(process_line, sample_human_df.iterrows()), total=len(sample_human_df)))

# t20_df = pd.concat([result for result in results if result is not None], ignore_index=True)
# Not_successful_index = [i for i, result in enumerate(results) if result is None]


save_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/t20_score_result.csv'
t20_df = pd.read_csv(save_fpath)
t20_df

,Raw_name,H_Score,H_gene,L_score,L-gene,H_seq,L_seq
0,3A3human_0,75.71437,vh,82.64157,vk,EVQLVESGGGLVQPGGSLRLSCVASGFTFSNFWMNWVRQAPGKGLE...,DIQMTQSPSPLSASVGDRANITCRASQDVSTDLGWYQQKPGKAPKV...
1,MAK195human_0,78.24797,vh,82.66367,vk,QVQLVESGGGVVQPGGSLRLSCAASGFSLTDYGVAWVRQPPGKGLE...,DIQMAQSPSSLSASVGDRVTITCRASQAVSSALTWYQQIPGKSPKL...
2,31#human_0,86.16677,vh,86.81427,vk,EVQLVQSGAEVKKPGESLKISCKASGYTFTDYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATISCKSSQSLLNTNSQKNYIAWYQQKP...
3,56#human_0,78.79037,vh,72.63897,vk,QVQLQESGPRLVEPSETLSLTCTVSGYSITSDYDWGWIRQPPGRGL...,DTLLTQSPDHLSVTLGERATISCRASQNIGTSLNWYQQKPGQSPKR...
4,74#human_0,78.24797,vh,85.30977,vk,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...
...,...,...,...,...,...,...,...
311,149G11human_0,77.30777,vh,88.30367,vk,EVRLVQSGAEVKKPGESLTISCKISGYTFTNYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATFNCKSSQNLLYNSNQKSYLAWYQQKP...
312,mu515H7human_0,86.54000,vh,79.33637,vk,EVQLVESGGGLVQPGGSLRLSCATSGFTFTDNYMDWVRQAPGKGLE...,IVLTQSPDSLAVSLGERATINCKSSQSLFNSRTRKNYLTWYQQKPG...
313,mu16B5human_0,63.97447,vh,76.68147,vk,QVRLVQSGAEVKKPGASVGVSCKASGFNIKDIYIHWVRQVPGQGPE...,DIVMTQFPNSLAVSLGERATINCKSSQSLLNSRTRKNYLSWYQKKA...
314,21B12human_0,87.03397,vh,90.53160,vk,QVQLVQSGSELKKPGASVKVSCKASGYTFTNYGLNWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATIDCKSSQSLLYSSNQKNYLTWYQQKP...


In [10]:
t20_filter_df = pd.DataFrame(columns=['Raw_name', 'H_Score', 'H_gene', 'L_score', 'L-gene', 'H_seq', 'L_seq'])
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     results = list(tqdm(executor.map(process_line, filter_human_df.iterrows()), total=len(filter_human_df)))

# t20_filter_df = pd.concat([result for result in results if result is not None], ignore_index=True)
# Not_successful_index = [i for i, result in enumerate(results) if result is None]
# save_filter_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/t20_score_filter_result.csv'
# t20_filter_df.to_csv(save_filter_fpath, index=False)

                        

                

In [16]:
lab_human_df = humanized_df[humanized_df['type'] != 'mouse'].reset_index(drop=True)
lab_human_df

,type,name,order_name,h_seq,l_seq
0,humanized,h3A3-5,0_humanized,EVQLVESGGGLVQPGGSLRLSCAASGFTFSNFWMDWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCKASQDVSTDVAWYQQKPGKAPKL...
1,humanized,AB240,1_humanized,EVQLQESGPGLVKPSETLSLTCTVSGGSISDYGVNWIRQPPGKGLE...,DIQMTQSPSSLSASVGDRVTITCKASQAVSSAVAWYQQKPGKAPKL...
2,humanized,31HZ,2_humanized,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTDYWMNWVRLAPGQGLE...,DVVITQSPDSLAVSLGERATINCKSSQSLLNTNSQKNYLAWYQQKP...
3,humanized,56HZ,3_humanized,QVQLQESGPGLVKPSQTLSLTCTVSGYFNSITSDYDWHWIRHHPGK...,DIVLTQSPATLSLSPGERATLSCRASQNIGTSIHWYQHKPGQSPRL...
4,humanized,74HZ,4_humanized,QVQLVQSGAEVKKPGSSVKVSCKASGYTFINYWMNWVRQAPGQGLE...,DIVITQSPDSLAVSLGERATINCKSSQTLLNSNTQKNYLAWYQQKP...
...,...,...,...,...,...
344,humanized,149G11-VH1VL1,350_humanized,QVQLVQSGAEVKKPGASVKVSCKASGYTFTNYWIHWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATINCKSSQNLLYNSNQKSYLAWYQQKP...
345,humanized,hz515H7 VH1 D76N VL2.1,352_humanized,EVQLVESGGGLVQPGRSLRLSCTASGFTFTDNYMSWVRQAPGKGLE...,DIVMTQSPDSLAVSLGERATMSCKSSQSLFNSRTRKNYLAWYQQKP...
346,humanized,hu16B5,353_humanized,QVQLVQSGAEVKKPGATVKISCKVSGFNIKDIYMHWVQQAPGKGLE...,DIVMTQSPDSLAVSLGERATINCKSSQSLLNSRTRKNYLAWYQQKP...
347,humanized,hu21B12,354_humanized,QVQLVQSGSELKKPGASVKVSCKASGYTFTNYGMHWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATINCKSSQSLLYSSNQKNYLAWYQQKP...


In [26]:
t20_lab_df = pd.DataFrame(columns=['Raw_name', 'H_score', 'H_gene', 'L_score', 'L-gene', 'H_seq', 'L_seq'])
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_line, lab_human_df.iterrows()), total=len(lab_human_df)))

t20_lab_df = pd.concat([result for result in results if result is not None], ignore_index=True)
Not_successful_index = [i for i, result in enumerate(results) if result is None]
save_lab_fpath = '/data/home/waitma/antibody_proj/antidiff/data/lab_data/t20_score_lab_filter_result.csv'
t20_lab_df.to_csv(save_lab_fpath, index=False)

  0%|                                                                                                                           | 0/349 [00:00<?, ?it/s]

Retry 1
HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vh&region=1&output=3&seqs=EVQLVQSGSELKKPGASVKVSCKASGYTFTNFGMNWVRQAPGQGLEWMGYINTNTGESIYSEEFKGRFVFSLDTSVSTAYLQICSLKAEDTAVYYCARSRMVTAYGMDYWGQGTTVTVSS (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vh&region=1&output=3&seqs=EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYRMHWVRQAPGQGLEWIGYINPSTGYTEYNQKFKDKATMTADKSISTAYMELSSLRSEDTAVYYCARGGGVFDYWGQGTLVTVSS (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))

Retry Retry1 
1
Retry 1
84.15977 92.92047
76.1296 79.72227
79.9586 87.89727


  0%|▎                                                                                                                  | 1/349 [00:07<44:39,  7.70s/it]

82.54177 89.27887
HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vh&region=1&output=3&seqs=EVQLVQSGAEVKKPGESLKISCKGSGYSFTGYTMNWVRQMPGKGLEWMGLINPYNGGTTYNQKFKGQVTISADKSISTAYLQWSSLKASDTAMYYCARDWDYYFDVWGQGTLVTVSS (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))
78.1256 87.07967
83.29067 87.14957


  1%|▋                                                                                                                  | 2/349 [00:09<24:30,  4.24s/it]

...

 89%|████████████████████████████████████████████████████████████████████████████████████████████████████▋            | 311/349 [05:26<00:23,  1.62it/s]

HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vh&region=1&output=3&seqs=EVQLVESGGGLVQPGGSLRLSCAASGFTFSDYGIHWVRQAPGKGLEWVAYISRGSSTIYYADTVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARGGYDTRNAMDYWGQGTTVTVSS (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))
Retry 1
84.29177 84.01877
77.66397 81.89197
77.39677 82.70277
65.90167 84.10717
66.52547 71.91597
HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vh&region=1&output=3&seqs=QVQLVQSGAEVKKPGASVKVSCKASGYTFASYYMHWMRQAPGQGLEWIGWINPGNVNTKYNEKFKGRATLTVDTSTNTAYMELSSLRSEDTAVYYCARSTYYRPLDYWGQGTLVTVSS (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))
73.64417 Retry70.85597 
1
83.71797 87.54467
84.755 88.36287


 89%|█████████████████████████████████████████████████████████████████████████████████████████████████████            | 312/349 [05:32<01:01,  1.67s/it]

75.21377 81.77577
HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vh&region=1&output=3&seqs=QVQLVQSGAEVKKPGASVKVSCKASGYTFTNYGMNWVRQARGQRLEWIGWINTYTGEPTYADDFKGRLTISKDTSKNQVVLTMTNMDPVDTATYYCTTYATSWYWGQG (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))
76.51267 84.15897
Retry 1
83.47117 85.14027
78.60477 77.83787
81.02467 87.94397
81.18647 92.56647
80.59837 87.25237
86.04177 92.34517
78.94967 84.11217
64.04557 77.82187
80.59837 81.58887
86.54477 86.30847
90.63567 91.91597
78.94077 82.92797
76.90687 82.80377
Retry 1
81.34157 87.94397
86.04177 92.34517
79.25627 86.20547
78.02427 87.94397
80.38467 84.73217
78.445 75.72737
HTTPSConnectionPool(host='sam.curiaglobal.com', port=443): Max retries exceeded with url: /t20/cgi-bin/blast.py?chain=vk&region=1&output=3&seqs=DVLMTQTPLTLSVTLGQPASISCKSSQSLLHSDGKTYLNWLQQRPGQSPKRLIYLVSKLDSGVPDRFTGSGSGTDFTLKISRVEAEDVGIYYCWQGTHLWTFGGGTKVEIK

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 349/349 [09:33<00:00,  1.64s/it]


In [ ]:
sns.kdeplot(t20_df['H_Score'], fill=True, label='VH')
sns.kdeplot(t20_df['L_score'], fill=True, label='VL')

plt.xlabel("Score")
plt.ylabel("Density")
plt.title("T20 Score Distributions")
plt.legend()
plt.show()

In [24]:
SCORE_REGEX = re.compile('<h3>The Z-score value of the Query sequence is: (-?[0-9.]+)</h3>')
def get_z_score_online(seq):
    chain = Chain(seq, scheme='imgt')
    chain_type = 'human_heavy' if chain.chain_type == 'H' else ('human_lambda' if chain.chain_type == 'L' else 'human_kappa')
    html = None
    for retry in range(5):
        url = f'http://www.bioinf.org.uk/abs/shab/shab.cgi?aa_sequence={seq}&DB={chain_type}'
        request = requests.get(url)
        time.sleep(0.5 + retry * 5)
        if request.ok:
            html = request.text
            break
        else:
            print('Retry', retry+1)
    if not html:
        raise ValueError('Z-score server is not accessible')
    matches = SCORE_REGEX.findall(html)
    if not matches:
        print(html)
        # raise ValueError(f'Error calling url {url}')
        return None, None
    return float(matches[0]), chain_type

def get_pair_data_zscore(h_seq, l_seq):
    h_z_score, h_type = get_z_score_online(h_seq)
    l_z_score, l_type = get_z_score_online(l_seq)
    return [h_z_score, h_type, l_z_score, l_type, h_seq, l_seq]

def process_z_score_line(line):
    h_seq = line[1]['h_seq']
    l_seq = line[1]['l_seq']
    name = [line[1]['name']]
    for retry in range(10):
        try:
            data = get_pair_data_zscore(h_seq, l_seq)
            if len(data) > 2:
                break
        except:
            time.sleep(5)
            continue
    if len(data) != 2:
        new_data = name + data
        new_line_df = pd.DataFrame([new_data], columns=zscore_filter_df.columns)
        return new_line_df
    else:
        return None

In [23]:
zscore_filter_df = pd.DataFrame(columns=['Raw_name', 'H_score', 'H_gene', 'L_score', 'L-gene', 'H_seq', 'L_seq'])

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_z_score_line, filter_human_df.iterrows()), total=len(filter_human_df)))

zscore_filter_df = pd.concat([result for result in results if result is not None], ignore_index=True)
Not_successful_index_z_score = [i for i, result in enumerate(results) if result is None]

save_filter_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/z_score_filter_result.csv'
zscore_filter_df.to_csv(save_filter_fpath)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:37<00:00,  3.11it/s]


In [25]:
zscore_lab_df = pd.DataFrame(columns=['Raw_name', 'H_score', 'H_gene', 'L_score', 'L-gene', 'H_seq', 'L_seq'])

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_z_score_line, lab_human_df.iterrows()), total=len(lab_human_df)))

zscore_lab_df = pd.concat([result for result in results if result is not None], ignore_index=True)
Not_successful_index_z_score = [i for i, result in enumerate(results) if result is None]

save_lab_fpath = '/data/home/waitma/antibody_proj/antidiff/data/lab_data/z_score_lab_filter_result.csv'
zscore_lab_df.to_csv(save_lab_fpath)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 349/349 [01:48<00:00,  3.22it/s]


In [10]:
save_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/z_score_result.csv'
zscore_df = pd.read_csv(save_fpath)

In [11]:
zscore_df

,Raw_name,H_Score,H_gene,L_score,L-gene,H_seq,L_seq
0,3A3human_0,-0.043,human_heavy,-0.023,human_kappa,EVQLVESGGGLVQPGGSLRLSCVASGFTFSNFWMNWVRQAPGKGLE...,DIQMTQSPSPLSASVGDRANITCRASQDVSTDLGWYQQKPGKAPKV...
1,MAK195human_0,0.792,human_heavy,-0.428,human_kappa,QVQLVESGGGVVQPGGSLRLSCAASGFSLTDYGVAWVRQPPGKGLE...,DIQMAQSPSSLSASVGDRVTITCRASQAVSSALTWYQQIPGKSPKL...
2,31#human_0,-0.142,human_heavy,-0.568,human_kappa,EVQLVQSGAEVKKPGESLKISCKASGYTFTDYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATISCKSSQSLLNTNSQKNYIAWYQQKP...
3,56#human_0,-1.133,human_heavy,-0.853,human_kappa,QVQLQESGPRLVEPSETLSLTCTVSGYSITSDYDWGWIRQPPGRGL...,DTLLTQSPDHLSVTLGERATISCRASQNIGTSLNWYQQKPGQSPKR...
4,74#human_0,-0.407,human_heavy,-0.267,human_kappa,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...
...,...,...,...,...,...,...,...
311,149G11human_0,-0.847,human_heavy,-0.385,human_kappa,EVRLVQSGAEVKKPGESLTISCKISGYTFTNYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATFNCKSSQNLLYNSNQKSYLAWYQQKP...
312,mu515H7human_0,0.612,human_heavy,-0.864,human_kappa,EVQLVESGGGLVQPGGSLRLSCATSGFTFTDNYMDWVRQAPGKGLE...,IVLTQSPDSLAVSLGERATINCKSSQSLFNSRTRKNYLTWYQQKPG...
313,mu16B5human_0,-1.985,human_heavy,-1.884,human_kappa,QVRLVQSGAEVKKPGASVGVSCKASGFNIKDIYIHWVRQVPGQGPE...,DIVMTQFPNSLAVSLGERATINCKSSQSLLNSRTRKNYLSWYQKKA...
314,21B12human_0,-0.432,human_heavy,-0.390,human_kappa,QVQLVQSGSELKKPGASVKVSCKASGYTFTNYGLNWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATIDCKSSQSLLYSSNQKNYLTWYQQKP...


In [ ]:
sns.kdeplot(zscore_df['H_Score'], fill=True, label='VH', color='gray')
sns.kdeplot(zscore_df['L_score'], fill=True, label='VL', color='cyan')

plt.xlabel("Score")
plt.ylabel("Density")
plt.title("Z-Score Distributions")
plt.legend()

In [35]:
zscore_df[zscore_df['L-gene'] == 'human_lambda']

,Raw_name,H_Score,H_gene,L_score,L-gene,H_seq,L_seq
245,18V4Fhuman_0,0.233,human_heavy,-2.322,human_lambda,QVPLVESGGGVVQPGKSLRLACAASGFSLTGYGMHWVRQAPGKGLE...,QTVVTQEPSLTVSPGGTVTFTCASSTGAVTTSNYPSWFQQKLGQAP...
249,8G8human_0,-0.919,human_heavy,-3.229,human_lambda,QVQLVQSGSELKNPGASVKVSCKAFGYTFTNYGINWVRQAPGQGLE...,LPVLTQPPAVSASLGASIKLTCTLSSQHSTYTIEWSQQRPERSPRF...
258,2D9human_0,0.398,human_heavy,-0.975,human_lambda,EVQVLESGGGSVQPGGSLRLSCAASGISLSSFAESWVRQAPGKGLE...,QTVVTQEPSVSVSPGGTVTLTCGLSTGAVSTSNYLSWYQQTPGQAP...
263,SP34human_0,0.076,human_heavy,-2.273,human_lambda,EVQLVESGGGLVQPGGSLTLSCAASGFTFNTYAMQWVRQASGKGLE...,QTVVTQEPSLTVSPGGTVTLTCASSTGAVTTSNYPNWLQQKPGQAP...
308,anti-CD3 antiboyhuman_0,-0.112,human_heavy,-1.816,human_lambda,EVQLVESGGGLVQPGESLRLSCVASGFTFNTYAMHWVRQASGKGLE...,QTVVTQEPSVTVSPGGTVTLTCGLSTGAVTTSNYPNWFQQKPGQAP...
315,mu19F6human_0,1.179,human_heavy,-0.095,human_lambda,EVQLVESGGALVQPGGSLRLSCTASGFTFSKYVMVWVRQAPGKGLE...,NLVLTQPHSVSGSPGKTVTISCTRSSGDIGDSYVHWYRQRPGSSPI...


In [24]:
test_chain

DIQMTQSPSSLSASVGDRVTITCRASQSISSYLNWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYSTPPTFGQGTRLEIK
                          ^^^^^^                 ^^^                                    ^^^^^^^^^          